# Practice Session 08: Data streams

Author: <font color="blue">Bernat Quintilla Castellón</font>

E-mail: <font color="blue">bernat.quintilla01@estudiant.upf.edu</font>

Date: <font color="blue">26/11/2023</font>

In [1]:
import io
import nltk
import gzip
import random
import statistics
import secrets
import re
import gzip

# 0. Dataset and how to iterate

In [2]:
# Leave this code as-is

INPUT_FILE = "movie_lines.tsv.gz"

In [3]:
# Leave this code as-is

# Producer in Python that reads a filename by words
def read_by_words(filename, max_words=-1, report_every=-1):
    
    # Open the input file
    with gzip.open(INPUT_FILE, "rt", encoding='utf8') as file:
        
        # Initialize counter of words to stop at max_words
        counter = 0
    
        # Regular expression to identify words having 3 letters or more and beginning with a-z
        word_expr = re.compile('^[a-z]{2,}$', re.IGNORECASE)

        # Iterate through lines in the file
        for line in file:
            
            elements = line.split("\t")
            
            text = ""
            if len(elements) >= 5:
                text = elements[4].strip()
                                        
            if counter > max_words and max_words != -1:
                break
                
            for word in nltk.word_tokenize(text):
                          
                if word_expr.match(word):
                    counter += 1
                    
                    # Report
                    if (report_every != -1) and (counter % report_every == 0):
                        if max_words == -1:
                            print("- Read %d words so far" % (counter))
                        else:
                            print("- Read %d/%d words so far" % (counter, max_words))

                    # Produce the word in lowercase
                    yield word.lower()

In [4]:
# Leave this code as-is

# Iterate through the file
for word in read_by_words(INPUT_FILE, max_words=300000, report_every=100000):
    # Prints 1/10000 of words
    if random.random() < 0.0001:
        print("Current word '%s'" % (word)) 

Current word 'may'
Current word 'cop'
Current word 'my'
Current word 'girl'
Current word 'do'
Current word 'the'
Current word 'not'
Current word 'talk'
Current word 'friend'
Current word 'to'
Current word 'it'
Current word 'something'
- Read 100000/300000 words so far
Current word 'be'
Current word 'involved'
Current word 'you'
Current word 'one'
Current word 'highest'
Current word 'most'
Current word 'you'
Current word 'he'
Current word 'told'
Current word 'if'
Current word 'camp'
- Read 200000/300000 words so far
Current word 'evening'
Current word 'of'
Current word 'begin'
Current word 'lydia'
Current word 'another'
Current word 'choice'
Current word 'marry'
Current word 'vector'
Current word 'he'
Current word 'just'
- Read 300000/300000 words so far


In [5]:
# Run this if above gives an error about 'punkt'
#nltk.download('punkt')

# 1. Determine approximately the top-10 words

<font size="+1" color="red">Replace this cell with your code for "add_reservoir"</font>

In [6]:
def add_to_reservoir(reservoir, item, max_reservoir_size):
    # YOUR CODE HERE
    if len(reservoir) < max_reservoir_size:
        reservoir.append(item)
    else:
        item_to_remove = random.choice(reservoir)
        reservoir.remove(item_to_remove)
        reservoir.append(item)
    assert(len(reservoir) <= max_reservoir_size)

<font size="+1" color="red">Replace this cell with your code for "reservoir_sampling"</font>

In [7]:
def reservoir_sampling(filename, reservoir_size, max_words=-1, report_every=-1):
    reservoir = []
    words_read = 0

    for word in read_by_words(filename, max_words=max_words, report_every=report_every):
        words_read += 1

        #Reservoir sampling
        if len(reservoir) < reservoir_size:
            #If reservoir is not full add word directly
            reservoir.append(word)
        else:
            #Reservoir is full
            #Probability of replacing an existing word is reservoir_size/words_read
            replace_probability = reservoir_size / words_read

            if random.random() < replace_probability:
                add_to_reservoir(reservoir, word, reservoir_size)

    return words_read, reservoir

In [8]:
# Leave this code as-is

reservoir_size = 1500
(items_seen, reservoir) = reservoir_sampling(INPUT_FILE, reservoir_size, max_words=1000000, report_every=100000)

print("Number of items seen    : %d" % items_seen)
print("Number of items sampled : %d" % len(reservoir) )

- Read 100000/1000000 words so far
- Read 200000/1000000 words so far
- Read 300000/1000000 words so far
- Read 400000/1000000 words so far
- Read 500000/1000000 words so far
- Read 600000/1000000 words so far
- Read 700000/1000000 words so far
- Read 800000/1000000 words so far
- Read 900000/1000000 words so far
- Read 1000000/1000000 words so far
Number of items seen    : 1000023
Number of items sampled : 1500


In [9]:
# Leave this code as-is

freq = {}
for item in reservoir:
    freq[item] = reservoir.count(item)

most_frequent_items = sorted([(frequency, word) for word, frequency in freq.items()], reverse=True)[:10]
for absolute_frequency, word in most_frequent_items:
    print("%d %s" % (absolute_frequency, word))

53 you
50 the
39 to
32 that
26 and
25 do
23 what
20 of
19 it
18 was


<font size="+1" color="red">Replace this cell with your code to print the top items and their relative frequencies</font>

In [10]:
#Compute total count of items in the reservoir
total_items = len(reservoir)

#Compute and print top 15 most frequent items and their relative frequencies
top_items = sorted([(frequency, word) for word, frequency in freq.items()], reverse=True)[:15]

for absolute_frequency, word in top_items:
    relative_frequency = (absolute_frequency / total_items) * 100
    print("%d %s (%.2f%%)" % (absolute_frequency, word, relative_frequency))


53 you (3.53%)
50 the (3.33%)
39 to (2.60%)
32 that (2.13%)
26 and (1.73%)
25 do (1.67%)
23 what (1.53%)
20 of (1.33%)
19 it (1.27%)
18 was (1.20%)
16 this (1.07%)
16 me (1.07%)
16 know (1.07%)
14 is (0.93%)
13 we (0.87%)


<font size="+1" color="red">Increase the max limit of words so that one pass takes no more than 5 minutes to be completed. Replace this cell with your code to try different reservoir sizes. In each case, print your estimate for the relative and absolute frequency of the words in the entire dataset.</font>

In [11]:
#Function to estimate the frequency in the entire dataset
def estimate_frequency(absolute_frequency, dataset_size, reservoir_size):
    return (absolute_frequency * dataset_size) / reservoir_size

reservoir_sizes = [50, 100, 500, 1000, 5000]

#Iterate over different reservoir sizes
for reservoir_size in reservoir_sizes:
    print(f"\nReservoir Size: {reservoir_size}\n{'=' * 40}")

    #Perform reservoir sampling
    words_read, reservoir = reservoir_sampling(INPUT_FILE, reservoir_size, max_words=300000, report_every=100000)

    #Compute total count of items in entire dataset
    total_dataset_size = words_read * (300000 / words_read)

    #Compute and print top 5 most frequent items and their estimates
    top_items = sorted([(frequency, word) for word, frequency in freq.items()], reverse=True)[:5]

    for absolute_frequency, word in top_items:
        relative_frequency = (absolute_frequency / words_read) * 100
        estimated_frequency = estimate_frequency(absolute_frequency, total_dataset_size, reservoir_size)

        print(f"{word}:")
        print(f"  Absolute Frequency: {absolute_frequency}")
        print(f"  Relative Frequency: {relative_frequency:.2f}%")
        print(f"  Estimated Frequency in Entire Dataset: {estimated_frequency:.2f}\n")


Reservoir Size: 50
- Read 100000/300000 words so far
- Read 200000/300000 words so far
- Read 300000/300000 words so far
you:
  Absolute Frequency: 53
  Relative Frequency: 0.02%
  Estimated Frequency in Entire Dataset: 318000.00

the:
  Absolute Frequency: 50
  Relative Frequency: 0.02%
  Estimated Frequency in Entire Dataset: 300000.00

to:
  Absolute Frequency: 39
  Relative Frequency: 0.01%
  Estimated Frequency in Entire Dataset: 234000.00

that:
  Absolute Frequency: 32
  Relative Frequency: 0.01%
  Estimated Frequency in Entire Dataset: 192000.00

and:
  Absolute Frequency: 26
  Relative Frequency: 0.01%
  Estimated Frequency in Entire Dataset: 156000.00


Reservoir Size: 100
- Read 100000/300000 words so far
- Read 200000/300000 words so far
- Read 300000/300000 words so far
you:
  Absolute Frequency: 53
  Relative Frequency: 0.02%
  Estimated Frequency in Entire Dataset: 159000.00

the:
  Absolute Frequency: 50
  Relative Frequency: 0.02%
  Estimated Frequency in Entire Datas

<font size="+1" color="red">Remove the max limit of words and re-run. Replace this cell with a brief commentary indicating what reservoir size you would recommend to use, and your overall conclusions.</font>

In [12]:
# Remove the max limit on words
max_words = -1

#Use of the same code as the previous cell of code
#Iterate over different reservoir sizes
for reservoir_size in reservoir_sizes:
    print(f"\nReservoir Size: {reservoir_size}\n{'=' * 40}")

    #Perform reservoir sampling
    words_read, reservoir = reservoir_sampling(INPUT_FILE, reservoir_size, max_words=max_words, report_every=100000)

    #Compute total count of items in entire dataset
    total_dataset_size = words_read * (300000 / words_read)

    #Compute and print top 5 most frequent items and their estimates
    top_items = sorted([(frequency, word) for word, frequency in freq.items()], reverse=True)[:5]

    for absolute_frequency, word in top_items:
        relative_frequency = (absolute_frequency / words_read) * 100
        estimated_frequency = estimate_frequency(absolute_frequency, total_dataset_size, reservoir_size)

        print(f"{word}:")
        print(f"  Absolute Frequency: {absolute_frequency}")
        print(f"  Relative Frequency: {relative_frequency:.2f}%")
        print(f"  Estimated Frequency in Entire Dataset: {estimated_frequency:.2f}\n")


Reservoir Size: 50
- Read 100000 words so far
- Read 200000 words so far
- Read 300000 words so far
- Read 400000 words so far
- Read 500000 words so far
- Read 600000 words so far
- Read 700000 words so far
- Read 800000 words so far
- Read 900000 words so far
- Read 1000000 words so far
- Read 1100000 words so far
- Read 1200000 words so far
- Read 1300000 words so far
- Read 1400000 words so far
- Read 1500000 words so far
- Read 1600000 words so far
- Read 1700000 words so far
- Read 1800000 words so far
- Read 1900000 words so far
- Read 2000000 words so far
- Read 2100000 words so far
- Read 2200000 words so far
- Read 2300000 words so far
- Read 2400000 words so far
- Read 2500000 words so far
- Read 2600000 words so far
- Read 2700000 words so far
- Read 2800000 words so far
- Read 2900000 words so far
you:
  Absolute Frequency: 53
  Relative Frequency: 0.00%
  Estimated Frequency in Entire Dataset: 318000.00

the:
  Absolute Frequency: 50
  Relative Frequency: 0.00%
  Estimat

For the given dataset, it seems that even a relatively small reservoir size (50 for example) provides some level of stability in the top words. A reservoir size around 50 or 100 is the one I would recommend. It provides stability in the top words while potentially ensuring faster processing within the one-hour time limit.

# 2. Determine approximately the distinct number of words

In [13]:
# Leave this code as-is

def count_trailing_zeroes(number):
    count = 0
    while number & 1 == 0:
        count += 1
        number = number >> 1
    return count

In [14]:
# Leave this code as-is

def random_hash_function():
    # We use a cryptographically safe generator for the salt of our hash function
    salt = secrets.token_bytes(32)
    return lambda string: hash(string + str(salt))

<font size="+1" color="red">Replace this cell with your code to perform the requested number of passes.</font>

In [15]:
number_of_passes = 5
estimates = []
for i in range(number_of_passes):
    #For each pass new hash function
    hash_function = random_hash_function()

    max_trailing_zeroes = 0

    #Iterate through lines in file
    with gzip.open(INPUT_FILE, "rt", encoding='utf8') as file:
        for line in file:
            elements = line.split("\t")
            text = ""
            if len(elements) >= 5:
                text = elements[4].strip()

            #Hash text with hash function
            hash_value = hash_function(text)

            #Count trailing zeroes
            trailing_zeroes = count_trailing_zeroes(hash_value)

            # Update maximum trailing zeros
            max_trailing_zeroes = max(max_trailing_zeroes, trailing_zeroes)

    #2^R as the estimate for the number of distinct elements
    estimate = 2 ** max_trailing_zeroes
    estimates.append(estimate)

    print("Estimate on pass %d: %d distinct words" % (i+1, estimate))

Estimate on pass 1: 65536 distinct words
Estimate on pass 2: 524288 distinct words
Estimate on pass 3: 1048576 distinct words
Estimate on pass 4: 65536 distinct words
Estimate on pass 5: 524288 distinct words


In [16]:
# Leave this code as-is

print("* Average of estimates: %.1f" % statistics.mean(estimates))
print("* Median of estimates: %.1f" % statistics.median(estimates))

* Average of estimates: 445644.8
* Median  of estimates: 524288.0


<font size="+1" color="red">Remove the limit of max words, or set to a high number, but notice that you do no need to use more than one hour of computer processing time, and perform the 10 passes. Replace this cell with the results you obtained in each pass, and whether the average or the median seem more appropriate for this probabilistic counting.</font>

In [26]:
#Use of similar cell as previous
number_of_passes = 20 #Set to 20
for idx in range(3): #Do 3 separate runs
    estimates = []
    print('Run ',idx+1,':')
    for i in range(number_of_passes):
        #For each pass new hash function
        hash_function = random_hash_function()

        max_trailing_zeroes = 0

        # Iterate through lines in file
        with gzip.open(INPUT_FILE, "rt", encoding='utf8') as file:
            for line in file:
                elements = line.split("\t")
                text = ""
                #Limit of max words removed
                text = elements[4].strip()

                #Hash text with hash function
                hash_value = hash_function(text)

                # Count trailing zeroes
                trailing_zeroes = count_trailing_zeroes(hash_value)

                # Update maximum trailing zeros
                max_trailing_zeroes = max(max_trailing_zeroes, trailing_zeroes)

        #2^R as the estimate for the number of distinct elements
        estimate = 2 ** max_trailing_zeroes
        estimates.append(estimate)

        print("Estimate on pass %d: %d distinct words" % (i+1, estimate))

    print("* Average of estimates: %.1f" % statistics.mean(estimates))
    print("* Median of estimates: %.1f" % statistics.median(estimates))
    print('')

Run  1 :
Estimate on pass 1: 4194304 distinct words
Estimate on pass 2: 262144 distinct words
Estimate on pass 3: 131072 distinct words
Estimate on pass 4: 1048576 distinct words
Estimate on pass 5: 262144 distinct words
Estimate on pass 6: 131072 distinct words
Estimate on pass 7: 131072 distinct words
Estimate on pass 8: 1048576 distinct words
Estimate on pass 9: 262144 distinct words
Estimate on pass 10: 2097152 distinct words
Estimate on pass 11: 262144 distinct words
Estimate on pass 12: 65536 distinct words
Estimate on pass 13: 131072 distinct words
Estimate on pass 14: 131072 distinct words
Estimate on pass 15: 262144 distinct words
Estimate on pass 16: 1048576 distinct words
Estimate on pass 17: 524288 distinct words
Estimate on pass 18: 262144 distinct words
Estimate on pass 19: 131072 distinct words
Estimate on pass 20: 262144 distinct words
* Average of estimates: 632422.4
* Median of estimates: 262144.0

Run  2 :
Estimate on pass 1: 524288 distinct words
Estimate on pass 2:

By running the code with 20 passes instead of 5 and without a limit on the maximum number of words, the median is more consistent across the 3 runs, in fact all the runs have 262144 as the median. The increased number of passes allows the allgorithm to have better convergence in the estimation process. So I would choose this option as the more appropriate for this probabilistic counting.

<font size="+2" color="#003300">I hereby declare that, except for the code provided by the course instructors, all of my code, report, and figures were produced by myself.</font>